In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
# from sklearn.cluster import HDBSCAN
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sentence_transformers import SentenceTransformer

2025-12-31 11:08:21.532421: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1767179301.798379      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1767179301.886268      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1767179302.592867      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767179302.592921      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1767179302.592924      55 computation_placer.cc:177] computation placer alr

In [45]:
!cp ../input/1300-towards-datascience-medium-articles-dataset/medium.csv /kaggle/working/.virtual_documents/medium.csv

In [ ]:
os.environ['WANDB_API_KEY'] = '81bfb0215bb0951acfe267a273ce29fb5b6d6edb'

In [2]:
f = pd.read_csv('/kaggle/input/1300-towards-datascience-medium-articles-dataset/medium.csv')

In [3]:
st = SentenceTransformer("all-MiniLM-L6-v2")

model = st[0].auto_model
tokenizer = st.tokenizer

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# levenshtein dist for chunking testing
def levenshtein_distance(str1, str2):
    len_str1, len_str2 = len(str1), len(str2)
    matrix = [[0] * (len_str2 + 1) for _ in range(len_str1 + 1)]
    
    for i in range(len_str1 + 1):
        matrix[i][0] = i
    for j in range(len_str2 + 1):
        matrix[0][j] = j

    for i in range(1, len_str1 + 1):
        for j in range(1, len_str2 + 1):
            if str1[i - 1] == str2[j - 1]:
                cost = 0 
            else:
                cost = 1
            
            matrix[i][j] = min(matrix[i - 1][j] + 1,       # Deletion
                               matrix[i][j - 1] + 1,       # Insertion
                               matrix[i - 1][j - 1] + cost) # Substitution
    return matrix[len_str1][len_str2]

# Example usage:
str1 = "kitten"
str2 = "sitting"
distance = levenshtein_distance(str1, str2)
print(f"The Levenshtein distance between '{str1}' and '{str2}' is {distance}.")


The Levenshtein distance between 'kitten' and 'sitting' is 3.


In [5]:
def tkn(x):
    t = tokenizer(x.replace('\n\n','\n').split('\n'),truncation=False)

    if max(list(map(len,t['input_ids'])))>255:
        t = tokenizer(x.split('. '),truncation=False)

        if max(list(map(len,t['input_ids'])))>255:
            return tokenizer(x.split(' '),truncation=False)['input_ids']
        else: 
            return t['input_ids']
    else: 
        return t['input_ids']

def agg_tkn(x):
    a=[[101,0]]    
    for n,i in enumerate(x):
        if len(a[-1][:-1]+i[1:])<128:
            a[-1][:-1]+=i[1:]
            a[-1] = a[-1][:-1]
        else:
            a[-1]=torch.tensor(a[-1])
            a+=[i]
    a[-1]=torch.tensor(a[-1])
    return a

def agg_tkn_rev(x):
    a=[[102,0]]
    for i in x[::-1]:
        if len(a[-1]+i[1:])<128:
            a[-1][:-1] += i[:-1][::-1]
            a[-1] = a[-1][:-1]
        else:
            a[-1]=torch.tensor(a[-1][::-1])
            a+=[i[::-1]]
    a[-1]=torch.tensor(a[-1][::-1])
    return a[::-1]

In [42]:
class PairedTextChunks2(Dataset):
    # chunks sampling from same doc not frequancy sensetive dataset  
    def remix(self): 
        # aggmentation of data based on shifitng text windows 
        b = random.choice([0,1,2,3])
        c = random.choice([0,1,2,3])
        if c==0:
            self.r = self.r.apply(lambda x:x[b:]).apply(agg_tkn)
        elif c==1:
            self.r = self.r.apply(lambda x:x[b:]).apply(agg_tkn_rev)
        elif c==2:
            self.r = self.r.apply(lambda x:x[:-b]).apply(agg_tkn_rev)
        else:
            self.r = self.r.apply(lambda x:x[:-b]).apply(agg_tkn)
        if ~self.singletones_on:
            self.f = self.r[self.r.apply(len)>5]
        else: self.f = self.r
        self.f = torch.cat(self.f.apply(
        lambda x:torch.nn.utils.rnn.pad_sequence(x+[torch.ones(256)]).T[:-1]).apply(
        lambda x:x[
        np.random.permutation(
        np.arange(len(x)//2*2)
        ).reshape(2,len(x)//2)]).tolist(),axis=1
        ).transpose(0,1)
        self.a = (self.f!=0).type(torch.float32)
        
    def __init__(self,pth, tr_ts_msk, singletones_on=False):
        # super.__init__()
        self.singletones_on = singletones_on
        self.r = pd.read_csv(pth)['Text'][tr_ts_msk].apply(tkn)
        self.remix() 
        
    def __len__(self): return len(self.f)
    def __getitem__(self, idx): return {'input_ids':self.f[idx],
                                        'token_type_ids':torch.zeros(256),
                                        'attention_mask':self.a[idx]}


tr_ts_msk = torch.bernoulli(torch.ones(len(f))*0.75).type(torch.bool).numpy()

tr_ds = PairedTextChunks2(
    tr_ts_msk = tr_ts_msk,
    pth = '/kaggle/input/1300-towards-datascience-medium-articles-dataset/medium.csv')
ts_ds = PairedTextChunks2(
    tr_ts_msk = ~tr_ts_msk,
    pth = '/kaggle/input/1300-towards-datascience-medium-articles-dataset/medium.csv')


/tmp/ipykernel_55/2696849932.py:14: DeprecationWarning: Bitwise inversion '~' on bool is deprecated and will be removed in Python 3.16. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  if ~self.singletones_on:
/tmp/ipykernel_55/2696849932.py:14: DeprecationWarning: Bitwise inversion '~' on bool is deprecated and will be removed in Python 3.16. This returns the bitwise inversion of the underlying int object and is usually not what you expect from negating a bool. Use the 'not' operator for boolean negation or ~int(x) if you really want the bitwise inversion of the underlying int.
  if ~self.singletones_on:


In [43]:
tr_data = DataLoader(tr_ds,batch_size=64)
ts_data = DataLoader(ts_ds,batch_size=64)

In [ ]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
epoches=10
optim_rate=1e-5
loss_fn  = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=optim_rate)
rsp = lambda x:x.view(x.shape[0]*2,x.shape[-1])
l = lambda x:[x[0]*x[1],x[2]]
norm = torch.nn.functional.normalize

In [ ]:
def nt_xent_loss_half(z, temperature=0.07):
    device = z.device
    N = z.size(0) // 2
    z1 = z[0::2]   # a_i
    z2 = z[1::2]   # b_i
    loss = 0.0

    for a, b in [(z1, z2), (z2, z1)]:
        logits = torch.matmul(a, z.T) / temperature  # (N, 2N)

        idx = torch.arange(N, device=device)
        logits[idx, 2*idx]   = -1e9
        logits[idx, 2*idx+1] = -1e9

        pos_logits = torch.sum(a * b, dim=1) / temperature

        max_logits = logits.max(dim=1, keepdim=True)[0]
        logsumexp = max_logits.squeeze() + \
                    torch.log(torch.exp(logits - max_logits).sum(dim=1))

        loss += -(pos_logits - logsumexp).mean()

    return loss / 2

In [ ]:
for epoch in range(epoches):
    for n,x in enumerate(tr_data):
        model.train()
        xx = x['input_ids'].view(l(x['input_ids'].shape)).to(device)
        aa = x['attention_mask'].view(l(x['attention_mask'].shape)).to(device)
        out = model(input_ids=xx,
                    token_type_ids=torch.zeros(xx.shape,dtype=torch.int64).to(device),
                    attention_mask=aa).last_hidden_state[:,0]
        out = norm(out, dim=-1)
        loss = nt_xent_loss_half(out, temperature=0.05)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if n%16==0:
            with torch.no_grad():
                model.eval()
                xx = x['input_ids'].view(l(x['input_ids'].shape)).to(device)
                aa = x['attention_mask'].view(l(x['attention_mask'].shape)).to(device)
                out = model(input_ids=xx,
                            token_type_ids=torch.zeros(xx.shape,dtype=torch.int64).to(device),
                            attention_mask=aa).last_hidden_state[:,0]
                out = norm(out, dim=-1)
                ts_loss = nt_xent_loss_half(out, temperature=0.05)
                print('Epoch:{} Batch:{} Train:{} Test:{}'.format(epoch,n//16,loss:.4f,ts_loss:.4f))


In [ ]:
art[0].to('cuda:0').shape,\
torch.zeros(art[0].shape,dtype=torch.int64).to('cuda:0').shape,\
(art[0]!=0).type(torch.int64).to('cuda:0').shape


In [ ]:
art = f['Text'][[0]].apply(tkn).apply(agg_tkn).apply(
        lambda x:torch.nn.utils.rnn.pad_sequence(x+[torch.ones(256)]).T[:-1])

out = model(input_ids=art[0].to('cuda:0'),
             token_type_ids=torch.zeros(art[0].shape,dtype=torch.int64).to('cuda:0'),
             attention_mask=(art[0]!=0).type(torch.int64).to('cuda:0')
            ).last_hidden_state[:,0]
out

In [ ]:
torch.round((norm(out)@norm(out).T)[torch.arange(14)+1,torch.arange(14)])